# day 12

https://adventofcode.com/12/day/12

In [ ]:
import logging
import logging.config
import os

import networkx as nx
import yaml

In [ ]:
with open('../logging.yaml') as fp:
    logging_config = yaml.load(fp, Loader=yaml.FullLoader)

logging.config.dictConfig(logging_config)

In [ ]:
FNAME = os.path.join('data', 'day12.txt')

LOGGER = logging.getLogger('day12')

## part 1

### problem statement:

#### loading data

In [ ]:
test_1 = """AAAA
BBCD
BBCC
EEEC"""

test_2 = """OOOOO
OXOXO
OOOOO
OXOXO
OOOOO"""

test_3 = """RRRRIICCFF
RRRRIICCCF
VVRRRCCFFF
VVRCCCJFFF
VVVVCJJCFE
VVIVCCJJEE
VVIIICJJEE
MIIIIIJJEE
MIIISIJEEE
MMMISSJEEE"""

In [ ]:
def load_data(fname=FNAME):
    with open(fname) as fp:
        return fp.read().strip()

In [ ]:
UP = -1j
DOWN = 1j
LEFT = -1
RIGHT = 1

def build_dict(data: str) -> dict[complex, str]:
    return {(j + i * 1j): c
            for (i, row) in enumerate(data.split('\n'))
            for (j, c) in enumerate(row)}


def build_graph(data: str) -> nx.Graph:
    d = build_dict(data)
    g = nx.Graph()

    g.add_nodes_from([(k, {'val': v}) for (k, v) in d.items()])

    all_nodes = g.nodes(data=True)
    for node, val_dict in all_nodes:
        for dir in [RIGHT, DOWN]:
            nbr_node = node + dir
            try:
                nbr_node_val = all_nodes[nbr_node]['val']
            except KeyError:
                continue

            if val_dict['val'] == nbr_node_val:
                g.add_edge(node, nbr_node)

    return g


def get_perimeter(component: set[complex], g: nx.Graph) -> int:
    return 4 * len(component) - 2 * g.subgraph(component).number_of_edges()


def get_score(component: set[complex], g: nx.Graph) -> int:
    return len(component) * get_perimeter(component=component, g=g)

g = build_graph(test_1)
# g.nodes(data=True)
# g.edges()

# list(nx.connected_components(g))

# for component in nx.connected_components(g):
#     break
#
# print(f"""
# {component = }
# {get_perimeter(component=component, g=g) = }
# """)

#### function def

In [ ]:
def q_1(data):
    g = build_graph(data)
    return sum(get_score(component, g) for component in nx.connected_components(g))

#### tests

In [ ]:
def test_q_1():
    LOGGER.setLevel(logging.DEBUG)
    assert q_1(test_1) == 140
    assert q_1(test_2) == 772
    assert q_1(test_3) == 1930
    LOGGER.setLevel(logging.INFO)

In [ ]:
test_q_1()

#### answer

In [ ]:
q_1(load_data())

## part 2

### problem statement:

In [ ]:
test_4 = """EEEEE
EXXXX
EEEEE
EXXXX
EEEEE"""

test_5 = """AAAAAA
AAABBA
AAABBA
ABBAAA
ABBAAA
AAAAAA"""

In [ ]:
g.degree

#### function def

In [ ]:
wl = 2+2j
pn = {wl + dir for dir in [UP, DOWN, LEFT, RIGHT]}
pn, sg.nodes()
pn, set(sg.nodes())

In [ ]:
import copy


def get_num_sides(component: set[complex], g: nx.Graph) -> int:
    sg = g.subgraph(component)
    all_edge_nodes = {n for (n, d) in sg.degree if d < 4}
    unseen_edge_nodes = copy.deepcopy(all_edge_nodes)
    num_sides = 0

    while unseen_edge_nodes:
        # walk along the edge and count sides
        walker_loc = unseen_edge_nodes.pop()

        # one edge case: a single node
        if len(unseen_edge_nodes) == 0:
            return 4

        # choose one direction that is _not_ an edge and one
        # direction that points to another edge
        found_walker_orientation = False
        for (out_dir, forward_dir) in ([LEFT, UP], [UP, RIGHT], [RIGHT, DOWN], [DOWN, LEFT]):
            out_nbr = walker_loc + out_dir
            forward_nbr = walker_loc + forward_dir
            if (out_nbr not in sg) and (forward_nbr in all_edge_nodes):
                found_walker_orientation = True
                break

        if not found_walker_orientation:
            raise ValueError(f"couldn't find walker orientation for {walker_loc = }")

        # start walking
        start_loc = walker_loc  # so we know when we return
        start_forward_dir = forward_dir
        num_sides = 1
        while True:
            # take step
            prev_walker_loc = walker_loc
            walker_loc += forward_dir

            unseen_edge_nodes.discard(walker_loc)

            # we finished a loop
            if (walker_loc == start_loc) and (forward_dir == start_forward_dir):
                break

            # check directions and turn until they're true
            while True:
                out_nbr = walker_loc + out_dir
                forward_nbr = walker_loc + forward_dir
                out_still_empty = (out_nbr not in sg) or (out_nbr == prev_walker_loc)
                forward_still_edge = forward_nbr in all_edge_nodes
                match out_still_empty, forward_still_edge:
                    case True, True:
                        # keep walking
                        break
                    case True, False:
                        # turn right
                        out_dir *= 1j
                        forward_dir *= 1j
                        num_sides += 1
                    case False, _:
                        # turn left
                        out_dir *= -1j
                        forward_dir *= -1j
                        num_sides += 1

    return num_sides

g = build_graph(test_1)
# for component in nx.connected_components(g):
#     val = {vd['val'] for (n, vd) in g.subgraph(component).nodes(data=True)}
#     print(f"{component = }")
#     print(f"{val = }")
#     print(f"{get_num_sides(component, g) = }")
#     print()
component = list(nx.connected_components(g))[2]
get_num_sides(component, g)

In [ ]:
def get_score(component: set[complex], g: nx.Graph) -> int:
    return len(component) * get_num_sides(component=component, g=g)

In [ ]:
def q_2(data):
    return False

#### tests

In [ ]:
def test_q_2():
    LOGGER.setLevel(logging.DEBUG)
    assert q_2(test_1) == 80
    assert q_2(test_2) == 436

    assert q_2(test_4) == 236
    assert q_2(test_5) == 368

    assert q_2(test_3) == 1206
    LOGGER.setLevel(logging.INFO)

In [ ]:
test_q_2()

#### answer

In [ ]:
q_2(load_data())

fin

# online solution

In [ ]:
from collections import deque

inp = []
with open(FNAME, 'r') as f:
  for line in f:
    inp.append(list(line.strip()))

# print(inp)
num_rows = len(inp)
num_cols = len(inp[0])

def in_bounds(rc):
  r, c = rc
  return (0 <= r < num_rows) and (0 <= c < num_cols)

def get_plant(rc):
  r, c = rc
  return inp[r][c]

def get_neighbors(rc):
  r, c = rc
  neighbors = []
  ds = [(-1, 0), (0, 1), (1, 0), (0, -1)] # NESW
  for (dr, dc) in ds:
    neighbors.append((r + dr, c + dc))
  return [n for n in neighbors if in_bounds(n)]

def get_plant_neighbors(rc):
  neighbors = get_neighbors(rc)
  return [n for n in neighbors if get_plant(n)==get_plant(rc)]

# BFS
def get_region(rc):
  visited = set()
  region = set()
  queue = deque([rc])
  while queue:
    node = queue.popleft()
    if node not in visited:
      visited.add(node)
      # visit node
      region.add(node)
      # add all unvisited neighbors to the queue
      neighbors = get_plant_neighbors(node)
      unvisited_neighbors = [n for n in neighbors if n not in visited]
      # print(f'At node {node}, ns: {neighbors}, unvisited: {unvisited_neighbors}')
      queue.extend(unvisited_neighbors)
  return region


def calc_edges(region):
  edges = 0
  for (r, c) in region:
    north_n = (r - 1, c) # TODO: Add const NORTH vector of (-1, 0)
    west_n = (r, c - 1)
    nw_n = (r - 1, c - 1)
    # TODO: Do this once, rotate 90 degrees and do it in a loop
    if (north_n not in region):
      # Top is an edge. But is it a new edge?
      # it's the same edge if the spot west of plot is in_bounds
      # and the NW plot is not the same plant (or is out of bounds)
      same_edge = (west_n in region) and (nw_n not in region)
      if not same_edge:
        edges += 1

    south_n = (r + 1, c)
    sw_n = (r + 1, c - 1)
    if south_n not in region:
      # bottom is an edge
      same_edge = (west_n in region) and (sw_n not in region)
      if not same_edge:
        edges += 1

    if west_n not in region:
      # left is an edge
      same_edge = (north_n in region) and (nw_n not in region)
      if not same_edge:
        edges += 1

    east_n = (r, c + 1)
    ne_n = (r - 1, c + 1)
    if east_n not in region:
      # right is an edge
      same_edge = (north_n in region) and (ne_n not in region)
      if not same_edge:
        edges += 1
  return edges


regions = []
visited = set()
for r in range(num_rows):
  for c in range(num_cols):
    rc = (r, c)
    if rc not in visited:
      region = get_region(rc)
      visited |= region
      regions.append(region)

# print(regions)

total_price = 0
for region in regions:
  plant = get_plant(next(iter(region)))
  area = len(region)
  edges = calc_edges(region)
  price = area * edges
  total_price += price
  # print(f'{plant} (area: {area}, edges: {edges}): {region}')

print(total_price)